In [0]:
%sql
CREATE OR REPLACE VIEW lakehouse.gold.most_valuable_client AS
SELECT 
  customer_sk,
  customer_name,
  segmento,
  pais,
  estado,
  cidade,
  
  -- Métricas de transações
  CAST(COUNT(*) AS DECIMAL(18,0)) AS total_transacoes,
  CAST(SUM(gross_value) AS DECIMAL(18,2)) AS valor_total,
  CAST(AVG(gross_value) AS DECIMAL(18,2)) AS ticket_medio,
  MIN(data_hora) AS primeira_transacao,
  MAX(data_hora) AS ultima_transacao,
  
  -- Frequência nos últimos 30 dias (baseado na data máxima da tabela)
  CAST(
    COUNT(
      CASE 
        WHEN data_hora >= (
          SELECT MAX(data_hora) 
          FROM lakehouse.silver.fact_transaction_revenue
        ) - INTERVAL 30 DAYS THEN 1 
      END
    ) AS DECIMAL(18,0)
  ) AS transacoes_ultimos_30_dias,
  
  -- Receita de taxas
  CAST(SUM(fee_revenue) AS DECIMAL(18,2)) AS comissao_total,
  
  -- Ranking por número de transações
  CAST(RANK() OVER (ORDER BY COUNT(*) DESC) AS DECIMAL(18,0)) AS ranking_por_transacoes,
  
  -- Classificação de cliente (Top 1, 2, 3 ou Outros)
  CASE 
    WHEN RANK() OVER (ORDER BY COUNT(*) DESC) = 1 THEN 'Top 1'
    WHEN RANK() OVER (ORDER BY COUNT(*) DESC) = 2 THEN 'Top 2'
    WHEN RANK() OVER (ORDER BY COUNT(*) DESC) = 3 THEN 'Top 3'
    ELSE 'Outros'
  END AS classificacao_cliente,
  
  -- Timestamp de cálculo
  current_timestamp() AS calculated_at

FROM lakehouse.silver.fact_transaction_revenue
GROUP BY 
  customer_sk,
  customer_name,
  segmento,
  pais,
  estado,
  cidade
ORDER BY total_transacoes DESC;
